<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

[ваш текст]

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) создайте явную реализации интерфейса и управление зависимостями 


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [1]:
using System;
using System.Collections.Generic;

// Интерфейсы для явной реализации
public interface IPaymentSecurity
{
    bool ValidateSecurity();
    void SetSecurityLevel(int level);
}

public interface IPaymentNotifications
{
    void SendReceipt(string transactionId);
    string GetNotificationChannel();
}

public interface IPaymentAnalytics
{
    void TrackPayment(decimal amount, string method);
    Dictionary<string, object> GetPaymentStats();
}

// Generic класс с управлением зависимостями
public class PaymentService<T> where T : PaymentMethod
{
    private readonly List<T> _payments;
    private readonly IPaymentNotifications _notifier;
    private readonly IPaymentAnalytics _analytics;

    public PaymentService(IPaymentNotifications notifier, IPaymentAnalytics analytics)
    {
        _payments = new List<T>();
        _notifier = notifier;
        _analytics = analytics;
    }

    public void AddPayment(T payment)
    {
        _payments.Add(payment);
        Console.WriteLine($"Добавлен: {payment.MethodName}");
        _analytics.TrackPayment(0, "PaymentAdded");
    }

    public void ProcessAllPayments(decimal amount)
    {
        Console.WriteLine($"Обработка всех платежей на сумму {amount}:");
        foreach (var payment in _payments)
        {
            payment.ProcessPayment(amount);
            _notifier.SendReceipt(Guid.NewGuid().ToString());
        }
        _analytics.TrackPayment(amount, "BatchProcessing");
    }
}

// Базовый класс
public class PaymentMethod
{
    public int PaymentMethodId { get; set; }
    public string MethodName { get; set; }
    public decimal MinAmount { get; set; }
    public string Currency { get; set; }
    public bool IsActive { get; set; }
    public DateTime CreatedDate { get; set; }
    public string Description { get; set; }
    public string Version { get; set; }
    public int Priority { get; set; }
    public string Category { get; set; }
    public decimal MaxAmount { get; set; }

    public PaymentMethod(int id, string name, decimal minAmount)
    {
        PaymentMethodId = id;
        MethodName = name;
        MinAmount = minAmount;
        Currency = "RUB";
        IsActive = true;
        CreatedDate = DateTime.Now;
        Description = "Базовый способ оплаты";
        Version = "1.0";
        Priority = 1;
        Category = "General";
        MaxAmount = 1000000;
    }

    public virtual void ProcessPayment(decimal amount)
    {
        if (ValidateAmount(amount))
        {
            Console.WriteLine($"Базовый платеж: {amount} {Currency}");
        }
    }

    public virtual bool ValidateAmount(decimal amount)
    {
        return amount >= MinAmount && amount <= MaxAmount;
    }

    public virtual string GetPaymentInfo()
    {
        return $"{MethodName} ({MinAmount}-{MaxAmount} {Currency})";
    }
}

// OnlinePayment с явной реализацией интерфейсов
public class OnlinePayment : PaymentMethod, IPaymentSecurity, IPaymentNotifications, IPaymentAnalytics
{
    public string PaymentUrl { get; set; }
    public string GatewayName { get; set; }
    public int TimeoutSeconds { get; set; }
    private int _securityLevel;
    private string _apiKey;
    private List<string> _transactionHistory;
    private decimal _totalProcessed;

    public OnlinePayment(int id, string name, decimal minAmount, string paymentUrl, string gatewayName)
        : base(id, name, minAmount)
    {
        PaymentUrl = paymentUrl;
        GatewayName = gatewayName;
        TimeoutSeconds = 30;
        _securityLevel = 1;
        _apiKey = "default_key";
        _transactionHistory = new List<string>();
        _totalProcessed = 0;
        Category = "Digital";
        MaxAmount = 500000;
    }

    // ИСПРАВЛЕНО: убраны вызовы интерфейсных методов
    public override void ProcessPayment(decimal amount)
    {
        if (ValidateAmount(amount) && ((IPaymentSecurity)this).ValidateSecurity())
        {
            Console.WriteLine($"Онлайн платеж: {amount} {Currency} через {GatewayName}");
            _totalProcessed += amount;
            _transactionHistory.Add($"Payment: {amount} {Currency}");
            ((IPaymentAnalytics)this).TrackPayment(amount, "Online");
            ((IPaymentNotifications)this).SendReceipt(Guid.NewGuid().ToString());
        }
    }

    // Явная реализация IPaymentSecurity
    bool IPaymentSecurity.ValidateSecurity()
    {
        Console.WriteLine($"Проверка безопасности уровня {_securityLevel}");
        return _securityLevel >= 1 && !string.IsNullOrEmpty(_apiKey);
    }

    void IPaymentSecurity.SetSecurityLevel(int level)
    {
        _securityLevel = level;
        Console.WriteLine($"Уровень безопасности установлен: {level}");
    }

    // Явная реализация IPaymentNotifications
    void IPaymentNotifications.SendReceipt(string transactionId)
    {
        Console.WriteLine($"Email квитанция отправлена: {transactionId}");
    }

    string IPaymentNotifications.GetNotificationChannel()
    {
        return "Email";
    }

    // Явная реализация IPaymentAnalytics
    void IPaymentAnalytics.TrackPayment(decimal amount, string method)
    {
        Console.WriteLine($"Аналитика: платеж {amount} методом {method}");
    }

    Dictionary<string, object> IPaymentAnalytics.GetPaymentStats()
    {
        return new Dictionary<string, object>
        {
            {"TotalProcessed", _totalProcessed},
            {"TransactionCount", _transactionHistory.Count},
            {"Gateway", GatewayName}
        };
    }
}

// BankTransfer с явной реализацией интерфейсов
public class BankTransfer : PaymentMethod, IPaymentSecurity, IPaymentNotifications
{
    public string BankData { get; set; }
    public decimal CommissionRate { get; set; }
    public string AccountNumber { get; set; }
    public string BIC { get; set; }
    private int _securityLevel;
    private List<decimal> _dailyTransactions;

    public BankTransfer(int id, string name, decimal minAmount, string bankData, 
                       decimal commissionRate, string accountNumber)
        : base(id, name, minAmount)
    {
        BankData = bankData;
        CommissionRate = commissionRate;
        AccountNumber = accountNumber;
        BIC = "044525225";
        _securityLevel = 2;
        _dailyTransactions = new List<decimal>();
        Category = "Bank";
        MaxAmount = 2000000;
    }

    public override void ProcessPayment(decimal amount)
    {
        decimal commission = amount * CommissionRate;
        decimal total = amount + commission;

        if (ValidateAmount(amount) && ((IPaymentSecurity)this).ValidateSecurity())
        {
            Console.WriteLine($"Банковский перевод: {amount} {Currency}");
            Console.WriteLine($"Комиссия: {commission} {Currency}, Итого: {total} {Currency}");
            _dailyTransactions.Add(amount);
            ((IPaymentNotifications)this).SendReceipt(Guid.NewGuid().ToString());
        }
    }

    // Явная реализация IPaymentSecurity
    bool IPaymentSecurity.ValidateSecurity()
    {
        Console.WriteLine($"Банковская безопасность уровня {_securityLevel}");
        return _securityLevel >= 2 && AccountNumber.Length >= 10;
    }

    void IPaymentSecurity.SetSecurityLevel(int level)
    {
        _securityLevel = level;
        Console.WriteLine($"Банковский уровень безопасности: {level}");
    }

    // Явная реализация IPaymentNotifications
    void IPaymentNotifications.SendReceipt(string transactionId)
    {
        Console.WriteLine($"SMS квитанция отправлена: {transactionId}");
    }

    string IPaymentNotifications.GetNotificationChannel()
    {
        return "SMS";
    }
}

// CashPayment с явной реализацией интерфейсов
public class CashPayment : PaymentMethod, IPaymentNotifications
{
    public string CashPickupPoint { get; set; }
    public string OperatorName { get; set; }
    private int _receiptCounter;
    private decimal _cashBalance;

    public CashPayment(int id, string name, decimal minAmount, string cashPickupPoint, string operatorName)
        : base(id, name, minAmount)
    {
        CashPickupPoint = cashPickupPoint;
        OperatorName = operatorName;
        _receiptCounter = 0;
        _cashBalance = 0;
        Category = "Cash";
        MaxAmount = 100000;
    }

    public override void ProcessPayment(decimal amount)
    {
        if (ValidateAmount(amount))
        {
            Console.WriteLine($"Оплата наличными: {amount} {Currency}");
            Console.WriteLine($"Пункт: {CashPickupPoint}, Оператор: {OperatorName}");
            _receiptCounter++;
            _cashBalance += amount;
            ((IPaymentNotifications)this).SendReceipt($"CASH-{_receiptCounter}");
        }
    }

    // Явная реализация IPaymentNotifications
    void IPaymentNotifications.SendReceipt(string transactionId)
    {
        Console.WriteLine($"Печать бумажной квитанции: {transactionId}");
    }

    string IPaymentNotifications.GetNotificationChannel()
    {
        return "PaperReceipt";
    }
}

// Реализации для внедрения зависимостей
public class EmailNotifier : IPaymentNotifications
{
    public void SendReceipt(string transactionId)
    {
        Console.WriteLine($"EMAIL: Квитанция {transactionId} отправлена");
    }

    public string GetNotificationChannel()
    {
        return "EmailService";
    }
}

public class AnalyticsService : IPaymentAnalytics
{
    private Dictionary<string, object> _stats = new Dictionary<string, object>();

    public void TrackPayment(decimal amount, string method)
    {
        Console.WriteLine($"АНАЛИТИКА: Отслеживание платежа {amount} методом {method}");
    }

    public Dictionary<string, object> GetPaymentStats()
    {
        return new Dictionary<string, object>(_stats);
    }
}

// Демонстрация
public class Program
{
    public static void Main()
    {
        
    }
}
Console.WriteLine("=== УПРАВЛЕНИЕ ЗАВИСИМОСТЯМИ И ЯВНЫЕ ИНТЕРФЕЙСЫ ===\n");

        var notifier = new EmailNotifier();
        var analytics = new AnalyticsService();
        var paymentService = new PaymentService<PaymentMethod>(notifier, analytics);

        var online = new OnlinePayment(1, "Интернет-оплата", 10, "https://pay.site.com", "Stripe");
        var bank = new BankTransfer(2, "Банковский перевод", 50, "Сбербанк", 0.015m, "40702810000000012345");
        var cash = new CashPayment(3, "Наличные", 1, "Касса магазина", "ИП Петров");

        paymentService.AddPayment(online);
        paymentService.AddPayment(bank);
        paymentService.AddPayment(cash);

        Console.WriteLine("1. ЯВНАЯ РЕАЛИЗАЦИЯ ИНТЕРФЕЙСОВ:");
        
        ((IPaymentSecurity)online).SetSecurityLevel(3);
        ((IPaymentSecurity)bank).SetSecurityLevel(2);

        Console.WriteLine("\n2. УПРАВЛЕНИЕ ЗАВИСИМОСТЯМИ:");
        paymentService.ProcessAllPayments(1000);

        Console.WriteLine("\n=== ПРОГРАММА ЗАВЕРШЕНА ===");

=== УПРАВЛЕНИЕ ЗАВИСИМОСТЯМИ И ЯВНЫЕ ИНТЕРФЕЙСЫ ===

Добавлен: Интернет-оплата
АНАЛИТИКА: Отслеживание платежа 0 методом PaymentAdded
Добавлен: Банковский перевод
АНАЛИТИКА: Отслеживание платежа 0 методом PaymentAdded
Добавлен: Наличные
АНАЛИТИКА: Отслеживание платежа 0 методом PaymentAdded
1. ЯВНАЯ РЕАЛИЗАЦИЯ ИНТЕРФЕЙСОВ:
Уровень безопасности установлен: 3
Банковский уровень безопасности: 2

2. УПРАВЛЕНИЕ ЗАВИСИМОСТЯМИ:
Обработка всех платежей на сумму 1000:
Проверка безопасности уровня 3
Онлайн платеж: 1000 RUB через Stripe
Аналитика: платеж 1000 методом Online
Email квитанция отправлена: 8c127abe-6f79-4205-880c-126104c40a22
EMAIL: Квитанция fbcb095c-4975-4663-ace9-904386039813 отправлена
Банковская безопасность уровня 2
Банковский перевод: 1000 RUB
Комиссия: 15.000 RUB, Итого: 1015.000 RUB
SMS квитанция отправлена: 1ce32747-41c9-47bf-949e-802879e43c8a
EMAIL: Квитанция 37d02dea-d5d4-4213-bab9-009d56e6f962 отправлена
Оплата наличными: 1000 RUB
Пункт: Касса магазина, Оператор: ИП Петро